# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

<br>

$p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T$ 
<br>

$A = ((1-\alpha)M + \frac{\alpha}{N}E)$
<br>

Opakujte: 
<br>

$\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [1]:
import csv
def read(file:str)->dict[list]:
    nodes = dict()
    with open(file,"r") as f:
        reader = csv.reader(f)
        for line in reader:
            line = [int(x)-1 for x in line]
            if line[0] in nodes:
                nodes[line[0]].append(line[1])
            else:
                nodes[line[0]] = [line[1]]
            if not line[1] in nodes:
                nodes[line[1]] = []
    return nodes
print(read("graph_1.txt"))
print(read("graph_2.txt"))


{0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: []}
{0: [1], 1: [2], 2: [3], 3: [4], 4: [0]}


In [2]:
import numpy as np
alph = 0.2
g1 = read("graph_1.txt")
p = np.ones((len(g1),1))/len(g1)

def genMx(g:dict[list])->np.ndarray:
    M = np.zeros((len(g),len(g)))
    for i in g:
        for j in g[i]:
            #if g[i] and g[j]:
            M[i,j] = 1/len(g[i])
    return M
def getA(g:dict[list],alph:int)->np.ndarray:
    M = genMx(g)
    E = np.ones((len(g),len(g)))
    a = np.zeros((len(g),1))
    for i in g:
        if len(g[i])==0:
            a[i] = 1
        else:
            a[i] = alph
    A = (1-a)*M+(a/len(g))*E
    return A
A = getA(g1,alph=alph)
print('A',A)
for i in range(100):
    p = np.transpose(A)@p

print(p)

A [[0.03333333 0.83333333 0.03333333 0.03333333 0.03333333 0.03333333]
 [0.03333333 0.03333333 0.83333333 0.03333333 0.03333333 0.03333333]
 [0.03333333 0.03333333 0.03333333 0.83333333 0.03333333 0.03333333]
 [0.03333333 0.03333333 0.03333333 0.03333333 0.83333333 0.03333333]
 [0.03333333 0.03333333 0.03333333 0.03333333 0.03333333 0.83333333]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
[[0.0656044 ]
 [0.11808792]
 [0.16007474]
 [0.19366419]
 [0.22053575]
 [0.242033  ]]


In [3]:
def process(name:str,numit:int = 0)->None:
    g = read(name)
    p = np.ones((len(g),1))/len(g)
    A = getA(g,alph=alph)
    #print('A',A)
    for i in range(numit):
        p = np.transpose(A)@p
    print(p)

process("graph_1.txt",100)
process("graph_2.txt",100)
process("graph_3.txt",100)
process("graph_4.txt",100)

[[0.0656044 ]
 [0.11808792]
 [0.16007474]
 [0.19366419]
 [0.22053575]
 [0.242033  ]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
[[0.17857143]
 [0.32142857]
 [0.32142857]
 [0.17857143]]
[[0.27257372]
 [0.15666713]
 [0.13837881]
 [0.10924643]
 [0.18531604]
 [0.06563464]
 [0.07218322]]


### Předpokládaný výstup

#### graph_1.xt
```python
array([[0.0656044 ],
       [0.11808792],
       [0.16007474],
       [0.19366419],
       [0.22053575],
       [0.242033  ]])
```
       
#### graph_2.txt
```python
array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])
```


#### graph_3.txt
```python
array([[0.17857143],
       [0.32142857],
       [0.32142857],
       [0.17857143]])
```


#### graph_4.txt
```python
array([[0.27257372],
       [0.15666713],
       [0.13837881],
       [0.10924643],
       [0.18531604],
       [0.06563464],
       [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [4]:
def init_HITS(g)-> tuple[np.ndarray,np.ndarray]:
    a0 = np.ones((len(g),1))
    h0 = np.ones((len(g),1))
    return a0,h0

In [5]:
def getA_HITS(g):
    M = np.zeros((len(g),len(g)))
    for i in g:
        for j in g[i]:
            #if g[i] and g[j]:
            M[i,j] = 1
    return M

In [6]:
def process_HITS(name:str,numit:int = 100):
    g = read(name)
    a,h = init_HITS(g)
    A = getA_HITS(g)
    #print(A)
    for i in range(numit):
        h1 = A@a
        h1 = h1/np.sum(h1)
        a = np.transpose(A)@h
        a = a / np.sum(a)
        h = h1
    print(a)
    print(h)
process_HITS("graph_1.txt",100)
process_HITS("graph_2.txt",100)
process_HITS("graph_3.txt",100)
process_HITS("graph_4.txt",100)



[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
[[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]


### Předpokládaný výstup

#### graph_1.xt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

### Předpokládaný výstup

```python
print(inverted_index['be'])
print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.